In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00


# Plant Disease Prediction

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import models,layers

# load data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
print(os.listdir("/content/drive/MyDrive/PlantVillage"))


['Potato___Late_blight', 'Potato___Early_blight', 'Potato___healthy']


In [7]:
IMG_SIZE=256
BATCH_SIZE=32
CHANNELS=3
EPOCHS=25

In [8]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/PlantVillage/",
    shuffle=True,
    image_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,

)

Found 2162 files belonging to 3 classes.


In [9]:
class_names=dataset.class_names

In [10]:
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [11]:
len(class_names)

3

In [12]:
plt.figure(figsize=(15,15))
for img_batch,label_batch in dataset.take(1):
    for i in range(12):
        #print(batch[i].shape)
        ax=plt.subplot(3,4,i+1)
        plt.imshow(img_batch[i].numpy().astype('uint8'))
        plt.axis('off')
        plt.title(class_names[ label_batch[i] ])

Output hidden; open in https://colab.research.google.com to view.

# 80%==>traindataset
# 20%==>testdataset
## 10%==>validation
## 10%==>test
    

In [13]:
l=len(dataset)

In [14]:
train_size=int(l*0.8)
train_size

54

In [15]:
train_dataset=dataset.take(train_size)

In [16]:
rem_dataset=dataset.skip(train_size)
rl=len(rem_dataset)

In [17]:
val_dataset=rem_dataset.take(int(rl*0.5))
len(val_dataset)

7

In [18]:
test_dataset=rem_dataset.skip(int(rl*0.5))
len(test_dataset)

7

## Let's wrap all this into a function

In [19]:
def get_train_test_val(dataset,train_split=0.8,val_split=0.1,shuffle=False):
    if shuffle:
        dataset=dataset.shuffle()
    ds_size=len(dataset)

    train_size=int(ds_size*train_split)
    val_size=int(ds_size*val_split)

    train_dataset=dataset.take(train_size)
    val_dataset=dataset.skip(train_size).take(val_size)
    test_dataset=dataset.skip(train_size+val_size)
    return train_dataset,val_dataset,test_dataset





In [20]:
train_ds,val_ds,test_ds=get_train_test_val(dataset)

In [21]:
len(val_ds)

6

In [22]:
train_ds=train_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(tf.data.AUTOTUNE)

In [23]:
from tensorflow import keras
from tensorflow.keras import layers


In [24]:
resize_and_rescale = keras.Sequential([
    layers.Resizing(IMG_SIZE, IMG_SIZE),
    layers.Rescaling(1/255),
])

In [25]:
DataAugumentaion=models.Sequential([
    layers.RandomFlip("horizontal"),

    layers.RandomContrast(0.2)
])

In [28]:
input_shape=(BATCH_SIZE,IMG_SIZE,IMG_SIZE,CHANNELS)
model=models.Sequential([
    resize_and_rescale,
    DataAugumentaion,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,CHANNELS)),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPool2D((2,2)),

    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(3,activation='softmax')

])
model.build(input_shape=input_shape)

In [29]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (32, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (32, 254, 254, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (32, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (32, 125, 125, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (32, 62, 62, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (32, 60, 60, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (32, 30, 30, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (32, 57600)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (32, 64)               │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (32, 3)                │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,742,979 (14.28 MB)

 Trainable params: 3,742,979 (14.28 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [31]:

IMG_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 10

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/PlantVillage/",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/PlantVillage/",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)


Found 2162 files belonging to 3 classes.
Using 1730 files for training.
Found 2162 files belonging to 3 classes.
Using 432 files for validation.


In [32]:

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels = []
for img, label in train_ds.unbatch():
    labels.append(label.numpy())

class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)


Class weights: {0: np.float64(0.7163561076604554), 1: np.float64(0.71993341656263), 2: np.float64(4.650537634408602)}


In [33]:

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)



In [34]:

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=[early_stopping]
)


Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 171s 3s/step - accuracy: 0.4924 - loss: 1.1080 - val_accuracy: 0.8333 - val_loss: 0.4466
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 159s 3s/step - accuracy: 0.8008 - loss: 0.3898 - val_accuracy: 0.8981 - val_loss: 0.2709
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 159s 3s/step - accuracy: 0.8983 - loss: 0.2126 - val_accuracy: 0.8171 - val_loss: 0.5049
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.8510 - loss: 0.3234 - val_accuracy: 0.9329 - val_loss: 0.1848
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.9351 - loss: 0.1591 - val_accuracy: 0.9606 - val_loss: 0.1039
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.9483 - loss: 0.1383 - val_accuracy: 0.9398 - val_loss: 0.1556
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.9386 - loss: 0.1311 - val_accuracy: 0.9491 - val_loss: 0.1533
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 158s 3s/step - accuracy: 0.9611 - loss: 0.1089 - val_accuracy: 0.9329 - v

In [35]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [37]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detects the TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ TPU initialized:", tpu.cluster_spec())
except ValueError:
    print("❌ TPU not found. Falling back to CPU/GPU.")
    strategy = tf.distribute.get_strategy()


❌ TPU not found. Falling back to CPU/GPU.


In [38]:

for images, labels in val_ds.take(1):
    preds = model.predict(images)
    predicted_labels = tf.argmax(preds, axis=1)
    print("Predicted:", predicted_labels.numpy())
    print("Actual:   ", labels.numpy())


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 796ms/step
Predicted: [0 1 1 2 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 2 1 0 1 1 0 0]
Actual:    [0 1 1 2 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 0 2 1 0 1 2 0 0]


In [39]:
# model.fit(train_ds,epochs=1,batch_size=BATCH_SIZE,verbose=1,validation_data=val_ds)

In [40]:
# model.fit(train_ds,epochs=10,batch_size=BATCH_SIZE,verbose=1,validation_data=val_ds)

In [41]:
model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 673ms/step - accuracy: 0.9876 - loss: 0.0503


[0.05507311597466469, 0.9834710955619812]

In [55]:
from tensorflow.keras.models import load_model

In [56]:
model.save("v2.h5")


TypeError: cannot pickle 'module' object

## model load

In [43]:
import json
with open("class_names.json", "w") as f:
    json.dump(train_ds.class_names, f)


In [44]:
import pickle

with open('/content/history.pkl', 'wb') as f:
    pickle.dump(history.history, f)


In [47]:
model=tf.keras.models.load_model('v1.h5')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'v1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [46]:
model.evaluate(test_ds)

8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 687ms/step - accuracy: 0.9876 - loss: 0.0456


[0.05507311597466469, 0.9834710955619812]

In [ ]:
scores=model.evaluate(test_ds)

In [ ]:
history=model.fit(train_ds,epochs=10,batch_size=BATCH_SIZE,verbose=1,validation_data=val_ds)

In [ ]:
version=1

In [ ]:
model.save(f"../models/{version}.h5")

In [ ]:
import os
v=max([int(i) for i in os.listdir("../models/")+['0']])+1

model.save(f"../models/{v}/{v}.h5")

In [ ]:
print(os.listdir("../models/"))